# Garbage Laplacians for Pascal-VOC with only fc layer

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')

from lib import models, graph, coarsening, utils
import tensorflow as tf
import numpy as np
import time
from scipy.sparse import csr_matrix
import scipy.sparse
from scipy.io import *
%matplotlib inline


In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', os.path.join('..', 'data', 'mnist'), 'Directory to store data.')

In [ ]:
# Number of Nearest Neighbors
k = 110

base_storage_dir = '/data4/abhijeet/Datasets/PASCAL_VOC/GCN/preprocessing/experiment3/'
temp_filename = base_storage_dir + 'scikit_fc7_k_' + str(k) + '.mat'

if os.path.exists(temp_filename):
    print('File already exists\n No Preprocessing Required')
else:
    print('Do preprocessing --> TODO --> call the fucntion with a specific parameter')
    
data = scipy.io.loadmat( temp_filename)

X_train          = data['X_train']
X_test           = data['X_test']
train_labels     = data['Y_train']
test_labels      = data['Y_test']
Adjacency_test   = data['Adjacency_test']
Adjacency_train  = data['Adjacency_train']

#delete unused variables
del data

In [ ]:
#define laplacians, coarsen adjacency, permutate data points.
train_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]
test_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]

train_perm ,test_perm = [] ,[]

for i in range(X_train.shape[0]):
    if not(i%1000):
        print(i)
    garbage_adjacency = csr_matrix(np.random.rand(k,k))
    graphs, temp_perm = coarsening.coarsen(garbage_adjacency, levels=FLAGS.coarsening_levels, self_connections=False)
    train_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        train_laplacians[j].append(temp_L[j])     

for i in range(X_test.shape[0]):
    if not(i%1000):
        print(i)
    garbage_adjacency = csr_matrix(np.random.rand(k,k))
    graphs, temp_perm = coarsening.coarsen(garbage_adjacency, levels=FLAGS.coarsening_levels, self_connections=False)
    test_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        test_laplacians[j].append(temp_L[j])     

#del Adjacency_test, Adjacency_train

In [ ]:
print(len(train_perm[0]),type(train_perm))
train_data = np.zeros((X_train.shape[0], len(train_perm[0]), X_train.shape[2]))
test_data = np.zeros((X_test.shape[0], len(test_perm[0]), X_test.shape[2]))
for i in range(train_data.shape[0]):
    if not(i%1000):
        print(i)
    train_data[i,:,:] = coarsening.perm_data_point(X_train[i,:,:], train_perm[i])

for i in range(test_data.shape[0]):
    if not(i%1000):
        print(i)
    test_data[i,:,:] = coarsening.perm_data_point(X_test[i,:,:], test_perm[i])

val_data = test_data.copy()
val_laplacians = test_laplacians.copy()
val_labels = test_labels.copy()

#del X_train, Y_train, X_test, Y_test 
#del perm_train, perm_test

In [ ]:
print(len(test_laplacians))
#print(test_laplacians[0])
L = []
for i in range(FLAGS.coarsening_levels):
    L.append(train_laplacians[i][0])
L = np.array(L)
for i in range(4):
    print(L[i].shape)

In [ ]:
from lib import models, graph, coarsening, utils
common = {}
common['dir_name']       = 'cifar10/'
common['num_epochs']     = 200
common['batch_size']     = 100
common['decay_steps']    = (train_data.shape[0] + val_data.shape[0]) / common['batch_size']
common['eval_frequency'] = 10 * common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
num_labels_per_image     = 2
C = train_labels.shape[1]  # number of classes
model_perf = utils.model_perf()

In [ ]:
if True:
    name = 'softmax'
    params = common.copy()
    params['dir_name'] += name
    params['regularization'] = 5e-4
    params['dropout']        = 0.5
    params['learning_rate']  = 0.2
    params['decay_rate']     = 0.95
    params['momentum']       = 0.9
    params['F']              = []
    params['F_0']            = 20
    params['K']              = []
    params['p']              = []
    params['M']              = [C] 
    params['train_laplacians'] = train_laplacians
    params['test_laplacians'] = val_laplacians
    params['val_laplacians'] = test_laplacians
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)